## Reporter: 路遥

# Generating Coherent Patterns of Activity from Chaotic Neural Networks. 
### D. Sussillo and L.F. Abbott, (2009) Neuron 63:544-557.

### Method:
- Stability Analysis of ODE
- Random Matrix Theory
- Recursive Least Square (RLS)

## 1. Model Description (courtesy to Larry Abbott)

<img  src="https://raw.githubusercontent.com/AStupidBear/Pictures/master/ForceLearning/fl_diagram.png"/>

## 2. Dynamics Analysis

### \begin{align*} & \dfrac {dx_{i}} {dt}=-x_{i}+\sum _{j}J_{ij}r_{j}+u_{i}z\\ & r_{j}=\tanh \left( x_{j}\right) \\ & z=\sum _{j}w_{j}r_{j}\end{align*} 


In [ ]:
gr(); using Interact

### Stability Analysis
$eig(I-J)$, $J_{ij}\sim g N\left( 0,\dfrac {1} {N}\right)$

In [ ]:
N =1000
@manipulate for g=0.4:0.2:1.5
J = g*1/√N*randn(N,N);
z = eigvals(J)
x = real.(z)
y = imag.(z)
scatter(x, y, xlims=[-2,2],ylims=[-2,2],leg=:none)
vline!([1])
end

### As $g$ surpass 1, the system gradually become chaotic instantaneously. 
H. Sompolinsky, A. Crisanti, and H. J. Sommers (1988) Phys. Rev. Lett. 61, 259

In [ ]:
function test(g)
    N = 1000
    Δt = 0.1
    T = 5000
    J = 1/√N*randn(N,N)
    x = 2*rand(N, T)-1
    for t = 1:T-1
        x[:,t+1] = (1-Δt)x[:,t] + Δt*g*J*tanh(x[:,t])
    end
    x[1,:]
end

for g =0.8:0.2:1.4
    plot(test(g)) |> display
end

### When the system is driven by $A \cos \left( \omega t\right)$, it behaves periodically.

In [ ]:
function test(A, ω)
    N = 1000
    Δt = 0.1
    T = 1000
    J = 1/√N*randn(N,N);
    g = 1.5
    ω = 1
    u = 2*rand(N)-1
    x = 2*rand(N, T)-1
    for t = 1:T-1
        x[:,t+1] = (1-Δt)x[:,t] + Δt*(g*J*tanh(x[:,t]) + u*A*cos(ω*t*Δt))
    end
    x[100,:]
end
@manipulate for A = 0:0.5:3, ω = 0:0.5:2
    plot(test(A, ω))
end

## 3. Force Learning

Define $\left\langle x\left( t\right) \right\rangle =\int _{0}^{T}x\left( t\right) dt$, denote $C=\left\langle rr^{T}\right\rangle$, $D=\left\langle fr\right\rangle $, $P=C^{-1}$

### \begin{align*} & E=\dfrac {1} {2}\int _{0}^{T}\left( z\left( t\right) -f\left( t\right) \right) ^{2}dt\\ & \dfrac {\partial E} {\partial w_{i}}=\sum _{j}C_{ij}w_{j}-D_{i}=0\\ & w=PD\end{align*} 

According to RLS,
### \begin{align*} & \Delta P=\dfrac {-Prr^{T}P} {1+r^{T}\Pr }\\ & \Delta D=fr\\ & \Delta w=\left( f-r^{T}w\right) P r \end{align*} 

We choose initial condition to be 

### $$P_{0}=\alpha I$$

<img  src="https://raw.githubusercontent.com/AStupidBear/Pictures/master/ForceLearning/derivition_1.png"/>
<img  src="https://raw.githubusercontent.com/AStupidBear/Pictures/master/ForceLearning/derivition_2.png"/>

In [ ]:
import ForceLearning
t = 0:0.1:1440
A = 1.3;
fr = 1/60;
f = (A/1.0)*sin(1π*fr*t) +
      (A/2.0)*sin(2π*fr*t) +
      (A/6.0)*sin(3π*fr*t) +
      (A/3.0)*sin(4π*fr*t)
f = f/1.5
@time net = ForceLearning.force(t, f)
z = net.records[:z]
plot([f z])

## 4. Relationship to RNN ( Recurrent Neural Networks)

- Structure is a subcategory of RNN
- Use RLS, not BPTT to train

##### <img  src="https://raw.githubusercontent.com/AStupidBear/Pictures/master/ForceLearning/rnn_fl_comparison.png"/>

## 5. Conclusions

- Pros
 - Easy to implement
 - Fast
 - Online, therefore more biologically plausible
- Cons
 - Less flexible and powerful compared to RNN
 - No long term memory compared to LSTM